In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import skimage.io as io
import skimage.transform as trans
import shutil

from skimage import data

In [ ]:
# ejecutar una sola vez
import shutil

# borramos los directorios demo creados hasta ahora para hacer las pruebas
shutil.rmtree('CASIA_Cropped')
shutil.rmtree('CASIA_Edge')
shutil.rmtree('CASIA_Segmented')
shutil.rmtree('CASIA_Segmented_v2')
shutil.rmtree('CASIA_v1')

In [100]:
# shutil.rmtree('CASIA-Iris-Segmented')
shutil.rmtree('U-net Output')

In [10]:
def create_segmented_dir(from_dir, to_dir):
    '''
    Crea la mimsma estructura con la que cuenta el dataset de CASIA en la que se guardará 
    las imágenes segmentadas.
    '''
    # creamos carpeta en la que guardaremos las imágenes
    os.mkdir(to_dir)
    for i in range(1, len(os.listdir(from_dir))+1): # 108 sujetos
        sample = to_dir + os.sep + "%.3d" % i
        os.mkdir(sample)
        for j in range(1,3): # 2 sesiones
            session = sample + os.sep + str(j)
            os.mkdir(session)
                   
    print("-> Estructura de directorios creados satisfactoriamente.")

create_segmented_dir('CASIA-IrisV1', 'CASIA-Iris-Segmented')

-> Estructura de directorios creados satisfactoriamente.


In [182]:
def get_paths(directory, last_element=False):
    '''
    Devuelve la ruta relativa de las muestras .bmp
    '''
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".bmp"):
                if last_element:
                    paths.append(str(file))
                else:
                    paths.append(os.path.join(root, file))
    return paths

In [197]:
original_paths = get_paths("CASIA-IrisV1") # paths de las muestras originales del dataset CASIA
samples_names = get_paths("CASIA-IrisV1",True) # nombre de las muestras y su extensión
original = [cv2.imread(muestra,0) for muestra in original_paths] # en blanco y negro
# plt.imshow(original[0]);

In [200]:
def testGenerator(originals, target_size= (320,320)):
    for muestra in originals:
        img = muestra / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,(1,)+img.shape)
        yield img

def saveResult(results, save_dir, names_results):
    '''
    Guarda los imágenes segmentadas resultado de la red preentrenada
    
    Parámetros:
    results -- resultado de la red preentrenada
    save_path -- ruta en la que se guardarán las muestras segmentadas
    name_path -- directorio del que cogeremos los nombres de las muestras
    
    flag_multi_class -- (de momento mantener pero no nos es útil)
    '''
    dim = (320, 280) # dimensiones de las muestras del dataset
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
        for i,item in enumerate(results):
            img = item[:,:,0]
            img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
            io.imsave(os.path.join(save_dir,names_results[i]),img)

In [101]:
# def testGenerator(directory, target_size = (320,320)):
#     '''
#     Genera las muestras que se le pasarán al método predecir de la red preentrenada.
    
#     Parámetros:
#     directory -- directorio dónde se encuentran las muestras a generar
#     num_image -- número máximo de imágenes que queremos generar
#     target_size -- shape de de la muestra
#     '''
# #     i = 0
#     for root, dirs, files in os.walk(directory):
#         if files: # si es directorio hoja
#             for file in files:
# #                 if i == num_image:
# #                     break # sólo queremos x imágenes para probar
#                 path_file = os.path.join(root,file)
#                 img = cv2.imread(path_file,0)
#                 img = img / 255
#                 img = trans.resize(img,target_size)
# #                 img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
#                 img = np.reshape(img,(1,)+img.shape)
# #                 i += 1
#                 yield img

# def get_samples_names(directory):
#     '''
#     Devuelve los nombres de las muestras(los que se encuentran en los directorios hoja)
    
#     Parámetros:
#     directory -- directorio de cuyos subdirectorios hoja se quiere el nombre 
#     '''
#     names = []
#     for root, dirs, files in os.walk(directory):
#         for file in files:
#             names.append(str(file))
            
#     return names

# def saveResult(results, save_dir, names_dir):
#     '''
#     Guarda los imágenes segmentadas resultado de la red preentrenada
    
#     Parámetros:
#     results -- resultado de la red preentrenada
#     save_path -- ruta en la que se guardarán las muestras segmentadas
#     name_path -- directorio del que cogeremos los nombres de las muestras
    
#     flag_multi_class -- (de momento mantener pero no nos es útil)
#     '''
#     dim = (320, 280) # dimensioned de las muestras del dataset
#     if not os.path.isdir(save_dir):
#         os.mkdir(save_dir)
#         names = get_samples_names(names_dir)
#     #     for item, name in zip(npyfile, names):
#     #         img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
#     #         cv2.imwrite(os.path.join(save_path,name),img) # con cv2 da problemas
#         for i,item in enumerate(results):
#             img = item[:,:,0]
#             img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
#             io.imsave(os.path.join(save_dir,names[i]),img)

In [191]:
from keras.models import load_model

model = load_model('Iris_unet_d5.h5')
# model.summary()

In [201]:
gen = testGenerator(original)
results = model.predict(gen,verbose=1)

756/756 [==============================] - 1018s 1s/step


In [207]:
# tiene que haber un directorio con las 756 muestras segmentadas
saveResult(results, "U-net Output", samples_names)

### Aplicamos los operadores morfológicos

Se harán sobre todas las muestras del directorio `U-net Output`

In [135]:
def morph_operator(images, save_dir):
#     if os.path.isdir(save_dir):
#         shutil.rmtree(save_dir)
#     os.mkdir(save_dir)
#     outputs = [os.path.join(images_dir,muestra) for muestra in os.listdir(images_dir)]
#     outputs_names = [muestra for muestra in os.listdir(images_dir)]
#     i = 0
    enhanced = []
    for img in images:
#         img = cv2.imread(output)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
        (thresh, binRed) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY) # se binariza
        opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=2) # se aplica operadores morfológicos(vigilar iteraciones)
        (thresh, binRed) = cv2.threshold(opening, 128, 255, cv2.THRESH_BINARY) # se binariza nuevamente
        # guardamos muestra mejorada
#         cv2.imwrite(os.path.join(save_dir, outputs_names[i]),binRed)
        enhanced.append(binRed)
        i = i + 1
    return enhanced

In [136]:
# outputs = [ os.path.join('U-net Output',muestra) for muestra in os.listdir('U-net Output')]
morph_operator("U-net Output", "U-net Output Enhanced")

In [137]:
def canny_edges(images_dir, save_dir):
    if os.path.isdir(save_dir):
        shutil.rmtree(save_dir)
    os.mkdir(save_dir)
    
    outputs = [os.path.join(images_dir,muestra) for muestra in os.listdir(images_dir)]
    outputs_names = [muestra for muestra in os.listdir(images_dir)]
    i = 0
    
    for output in outputs:
        img = cv2.imread(output)
        edges = cv2.Canny(img, 10, 255)
        # guardamos muestra mejorada
        cv2.imwrite(os.path.join(save_dir, outputs_names[i]),edges)
        i = i + 1

In [138]:
canny_edges("U-net Output Enhanced", "U-net Output Edges")

In [ ]:
from skimage import data
from skimage.transform import (hough_line, hough_line_peaks, hough_circle, hough_circle_peaks)
from skimage.draw import circle_perimeter
from skimage.color import gray2rgb
from matplotlib import cm

def draw_circles(img, cx, cy, radii):
    '''
    A partir de los centros y el radio detectados dibuja el iris sobre la imagen que se le
    pasa como parámetro.
    '''
#     image = gray2rgb(img) # convertimos a color
#     for center_y, center_x, radius in zip(cy, cx, radii):
#         circy, circx = circle_perimeter(center_y, center_x, radius)
# #         print(image[circy,circx])
#         image[circy, circx] = (0, 255, 0) # green
    image = img
    pupil = cv2.circle(image,(cx[0],cy[0]), radii[0], (255, 0, 0), 2)
    iris = cv2.circle(image,(cx[1],cy[1]), radii[1], (255, 0, 0), 2)
    return image

In [145]:
dir_original = "CASIA-IrisV1"
dir_edges = "U-net Output Edges"
original = get_samples_names(dir_original)
print(original[0])
original = [cv2.imread(os.path.join(dir_original,o)) for o in original]
edges = [cv2.imread(os.path.join(dir_edges,muestra),0) for muestra in os.listdir(dir_edges)]

001_1_1.bmp


In [ ]:
def get_centers_radius(images_dir):
    outputs = [os.path.join(images_dir,muestra) for muestra in os.listdir(images_dir)]
    outputs_names = [muestra for muestra in os.listdir(images_dir)]
    
    boundaries = dict() # contiene los bordes límbico y pupilar
    centers = dict() # contiene centros y radio de la pupila
    i = 0
    for output in outputs:
        gray_img = cv2.imread(output,0)
        hough_radii = np.arange(20, 150, 2) # ajustar manualmente para que empiece desde un rango que incluya tanto al iris como a la pupila
        hough_res = hough_circle(gray_img, hough_radii)
        
        accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=2) # total_num_peaks -> nro de círculos
        x_pupil, x_iris = cx
        y_pupil, y_iris = cy
        radius_pupil, radius_iris = radii
        boundaries[outputs_names[i]] = 
        centers[outputs_names[i]]['pupil'] = [x_pupil, y_pupil, radius_pupil]
        centers[outputs_names[i]]['iris'] = [x_iris, y_iris, radius_iris]
        
        i = i + 1
sample = cv2.imread(path_sample,0)
gray_img = cv2.imread(path_edged, 0)

boundaries = [] # contiene los bordes límbico y pupilar
centers = [] # contiene centros y radio de la pupila

hough_radii = np.arange(20, 150, 2) # ajustar manualmente para que empiece desde un rango que incluya tanto al iris como a la pupila
hough_res = hough_circle(gray_img, hough_radii)

# ¡Ojo! Seleccionamos los círculos más prominentes, en este caso sabemos a priori que sólo puede haber 2, si subimos
# esa cantidad detectará círculos en zonas aleatorias
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=2) # total_num_peaks -> nro de círculos
x_pupil, x_iris = cx
y_pupil, y_iris = cy
radius_pupil, radius_iris = radii

print("center of pupil: ", (x_pupil,y_pupil))
print("radius of pupil: ", radius_pupil)
print("center of iris: ", (x_iris,y_iris))
print("radius of iris: ", radius_iris)
print("-------------------------")

iris_sample = draw_circles(sample, cx, cy, radii)
iris_edged = draw_circles(gray_img, cx, cy, radii) 

# bordes del iris 
boundaries.append(iris_sample)
# centros y radios de la pupila
centers.append([x_pupil, y_pupil, radius_pupil])

In [152]:
def get_samples(directory):
    '''
    Devuelve los nombres de las muestras(los que se encuentran en los directorios hoja)
    
    Parámetros:
    directory -- directorio de cuyos subdirectorios hoja se quiere el nombre 
    '''
    names = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            names.append(cv2.imread(file))
            
    return names
l = get_samples("CASIA-IrisV1")


In [166]:

l = get_paths("CASIA-IrisV1")